## README: This is the final notebook. Use this notebook to paste the final code.

### Pre Processing of Tweets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from datetime import timedelta, datetime
from dateutil import parser
import re

In [ ]:
tweets = pd.read_csv('raw_trump_tweets.csv')

In [ ]:
# >= 20th jan 2017, <= 31st Dec 2018
tweets['date']= pd.to_datetime(tweets['date'])
tweets = tweets[tweets['date'] >= np.datetime64('2017-01-20')]
tweets = tweets[tweets['date'] <= np.datetime64('2018-12-31')]


In [ ]:
def process_and_remove_duplicates(tweets, date):
    duplicated_rows = tweets[tweets['date'] == date]
    id = max(duplicated_rows['id'])
    max_favorite = max(duplicated_rows['favorites'])
    max_retweet = max(duplicated_rows['retweets'])
    combined_tweet = ""
    device = duplicated_rows['device'].iloc[0]
    for index, row in duplicated_rows.iterrows():
        combined_tweet = row['text'] + ' ' + combined_tweet.strip('.')
    tweets = tweets[tweets.date != date]
    row = {'id': id, 'text': combined_tweet, 'isRetweet': 'f', 'isDeleted': 'f', 'device': device, 'favorites': max_favorite, 'retweets': max_retweet, 'date': date}
    tweets = tweets.append(row, ignore_index = True)   
    print(f"Combined {len(duplicated_rows)} tweets on {date}")
    return tweets

In [ ]:
dates = tweets["date"]
duplicated = tweets[dates.isin(dates[dates.duplicated()])]
duplicated_dates = duplicated['date'].unique()

for date in duplicated_dates:
    tweets = process_and_remove_duplicates(tweets, date)

In [ ]:
## tweets that are Trump's
tweets1 = tweets[tweets['isRetweet'] == 'f'].reset_index(drop=True)
tweets1 = tweets1[tweets1['text'].str.contains('RT @') == False]
tweets1['cleaned_text'] = tweets1['text'].apply(lambda x: re.sub(r'https?:\/\/\S*', '', x, flags=re.MULTILINE))
tweets1 = tweets1[tweets1['cleaned_text'].str.strip() != '']

tweets2 = tweets1.loc[:, ['id', 'cleaned_text', 'favorites', 'retweets', 'date']]
tweets2['date_new'] = [i + timedelta(hours = 8) for i in tweets2['date']]
tweets2['date_part'] = [i.date() for i in tweets2['date_new']]
tweets2['time_part'] = [i.time() for i in tweets2['date_new']]
tweets2['hour'] = [int(str(i).split(":")[0]) for i in tweets2['time_part']]
tweets2['year'] = [int(str(i).split("-")[0]) for i in tweets2['date_part']]
tweets2['month'] = [int(str(i).split("-")[1]) for i in tweets2['date_part']]


### Pre Processing of Stock Data

In [ ]:
# Datasets are combined after pre processing! (else kernel crashes)
stock_price_2017 = pd.read_csv('yr2017.csv')
stock_price_2018 = pd.read_csv('yr2018.csv')

In [ ]:
# Drop unwanted columns 
stock_price_2017.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)
stock_price_2018.drop(columns=['SYM_ROOT', 'SYM_SUFFIX'], inplace=True)

In [ ]:
# Change to datetime objects
stock_price_2017['TIME_M'] = pd.to_datetime(stock_price_2017['TIME_M'], format='%H:%M:%S.%f')
stock_price_2018['TIME_M'] = pd.to_datetime(stock_price_2018['TIME_M'], format='%H:%M:%S.%f')

In [ ]:
# Remove seconds, milliseconds from data
stock_price_2017['TIME_M'] = stock_price_2017['TIME_M'].dt.floor('Min').dt.time
stock_price_2018['TIME_M'] = stock_price_2018['TIME_M'].dt.floor('Min').dt.time

In [ ]:
# Get minute to minute data (one entry for each minute)
stock_price_2017.drop_duplicates(subset=['TIME_M','DATE'],inplace=True)
stock_price_2018.drop_duplicates(subset=['TIME_M', 'DATE'],inplace=True)

In [ ]:
# Export to csv, will be used for preprocessing later!
stock_price = pd.concat([stock_price_2017,stock_price_2018], ignore_index=True, sort=False)
stock_price.to_csv("stock_price.csv", index=False, header=True)